In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
df = pd.read_csv('attribution_allocation_student_data.csv')
df.head()

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier
0,True,email,referral,referral,NaN,NaN,1
1,True,referral,referral,email,NaN,NaN,1
2,True,referral,NaN,NaN,NaN,NaN,1
3,True,email,NaN,NaN,NaN,NaN,1
4,True,referral,referral,social,referral,NaN,1


In [3]:
tier1=df.loc[df['tier'] == 1]
tier2=df.loc[df['tier'] == 2]
tier3=df.loc[df['tier'] == 3]

In [4]:
convert1=df.loc[(df['tier'] == 1)&(df['convert_TF']==True)]
convert2=df.loc[(df['tier'] == 2)&(df['convert_TF']==True)]
convert3=df.loc[(df['tier'] == 3)&(df['convert_TF']==True)]
convert1.index = range(len(convert1))
convert2.index = range(len(convert2))
convert3.index = range(len(convert3))
convert1

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier
0,True,email,referral,referral,NaN,NaN,1
1,True,referral,referral,email,NaN,NaN,1
2,True,referral,NaN,NaN,NaN,NaN,1
3,True,email,NaN,NaN,NaN,NaN,1
4,True,referral,referral,social,referral,NaN,1
...,...,...,...,...,...,...,...
2819,True,paid_search,referral,referral,referral,NaN,1
2820,True,social,referral,referral,NaN,NaN,1
2821,True,referral,NaN,NaN,NaN,NaN,1
2822,True,referral,referral,referral,social,organic_search,1


In [5]:
convert1.loc[352]

convert_TF        True
touch1        referral
touch2        referral
touch3           email
touch4             NaN
touch5             NaN
tier                 1
Name: 352, dtype: object

In [6]:
def get_first(d):
    first=[]
    b=len(d)
    for n in range(0,b):
        a=d.loc[n]
        first.append(a['touch1'])
    return first

f=get_first(convert1)

In [7]:
def get_last(d):
    last=[]
    b=len(d)
    for n in range(0,b):
        a=d.loc[n]
        if pd.isna(a['touch5']):
            if pd.isna(a['touch4']):
                if pd.isna(a['touch3']):
                    if pd.isna(a['touch2']):
                        last.append(a['touch1'])
                    else:
                        last.append(a['touch2'])
                else:
                    last.append(a['touch3'])
            else:
                last.append(a['touch4'])
        else:
            last.append(a['touch5'])
    return last

c=get_last(convert1)

In [10]:
def get_last_nondirect(d):
    last=[]
    b=len(d)
    for n in range(0,b):
        a=d.loc[n]
        if (pd.isna(a['touch5']))or(a['touch5']=='direct'):
            if pd.isna(a['touch4'])or(a['touch4']=='direct'):
                if pd.isna(a['touch3'])or(a['touch3']=='direct'):
                    if pd.isna(a['touch2'])or(a['touch2']=='direct'):
                        if pd.isna(a['touch1'])or(a['touch1']=='direct'):
                            last.append('direct')
                        else:
                            last.append(a['touch1'])
                    else:
                        last.append(a['touch2'])
                else:
                    last.append(a['touch3'])
            else:
                last.append(a['touch4'])
        else:
            last.append(a['touch5'])
    return last
v=get_last_nondirect(convert1)

In [11]:
e=pd.Series(v)
e.value_counts()

referral          1189
social             622
display            531
email              299
paid_search        180
organic_search       2
direct               1
dtype: int64

In [12]:
df2 = pd.read_csv('channel_spend_student_data.csv')
df2.head()

,Unnamed: 0,0
0,tier1,"{'email': 1000.0, 'social': 1000.0000000000001..."
1,tier2,"{'email': 2000.0, 'social': 2000.0, 'display':..."
2,tier3,"{'email': 3000.0, 'social': 3000.0, 'display':..."


In [105]:
#get First interaction method channel CAC
    
tier1=eval(df2.iloc[0,1])
tier2=eval(df2.iloc[1,1])
tier3=eval(df2.iloc[2,1])
tier_key=list(tier1.keys())
tier1_CAC=[]
tier2_CAC=[]
tier3_CAC=[]
average_chanel=[]

for i in tier_key:
    try:
        cac=tier1[i]/list(get_first(convert1)).count(i)
        tier1_CAC.append(cac)
    except:
        cac=0
        tier1_CAC.append(cac)
    

for i in tier_key:
    try:
        cac=tier2[i]/list(get_first(convert2)).count(i)
        tier2_CAC.append(cac)
    except:
        cac=0
        tier2_CAC.append(cac)    
    

for i in tier_key:
    try:
        cac=tier3[i]/list(get_first(convert3)).count(i)
        tier3_CAC.append(cac)
    except:
        cac=0
        tier3_CAC.append(cac)
        
for i in range(len(tier_key)):
    ave=(tier1_CAC[i]+tier2_CAC[i]+tier3_CAC[i])/3
    average_chanel.append(ave)

In [106]:
First_CAC=pd.DataFrame([tier1_CAC,tier2_CAC,tier3_CAC,average_chanel],
                       index=['tier1','tier2','tier3','average'],
                       columns=tier_key)
First_CAC.to_csv('First_CAC.csv')
print(First_CAC)

            email    social   display  paid_search  referral  organic_search  \
tier1    4.065041  2.123142  2.066116     7.812500  0.669792             0.0   
tier2    4.651163  2.506266  2.721088     8.097166  0.747943             0.0   
tier3    5.093379  3.036437  3.571429     9.316770  0.905524             0.0   
average  4.603194  2.555282  2.786211     8.408812  0.774420             0.0   

         direct  
tier1       0.0  
tier2       0.0  
tier3       0.0  
average     0.0  


In [107]:
#get Last interaction method channel CAC
    
tier1=eval(df2.iloc[0,1])
tier2=eval(df2.iloc[1,1])
tier3=eval(df2.iloc[2,1])
tier_key=list(tier1.keys())
tier1_CAC=[]
tier2_CAC=[]
tier3_CAC=[]
average_chanel=[]

for i in tier_key:
    try:
        cac=tier1[i]/list(get_last(convert1)).count(i)
        tier1_CAC.append(cac)
    except:
        cac=0
        tier1_CAC.append(cac)
    
for i in tier_key:
    try:
        cac=tier1[i]/list(get_last(convert2)).count(i)
        tier2_CAC.append(cac)
    except:
        cac=0
        tier2_CAC.append(cac)
        
for i in tier_key:
    try:
        cac=tier3[i]/list(get_last(convert3)).count(i)
        tier3_CAC.append(cac)
    except:
        cac=0
        tier3_CAC.append(cac)
        
for i in range(len(tier_key)):
    ave=(tier1_CAC[i]+tier2_CAC[i]+tier3_CAC[i])/3
    average_chanel.append(ave)

In [108]:
Last_CAC=pd.DataFrame([tier1_CAC,tier2_CAC,tier3_CAC,average_chanel],
                      index=['tier1','tier2','tier3','average'],
                      columns=tier_key)
Last_CAC.to_csv('Last_CAC.csv')
print(Last_CAC)

            email    social   display  paid_search  referral  organic_search  \
tier1    3.344482  1.607717  1.883239     5.555556  0.842460             0.0   
tier2    1.879699  0.935454  1.213592     2.941176  0.470810             0.0   
tier3    4.373178  2.388535  3.128259     6.198347  1.123596             0.0   
average  3.199120  1.643902  2.075030     4.898360  0.812288             0.0   

         direct  
tier1       0.0  
tier2       0.0  
tier3       0.0  
average     0.0  


In [109]:
#get Last non-direct click method channel CAC
tier1=eval(df2.iloc[0,1])
tier2=eval(df2.iloc[1,1])
tier3=eval(df2.iloc[2,1])
tier_key=list(tier1.keys())
tier1_CAC=[]
tier2_CAC=[]
tier3_CAC=[]
average_chanel=[]

for i in tier_key:
    try:
        cac=tier1[i]/list(get_last_nondirect(convert1)).count(i)
        tier1_CAC.append(cac)
    except:
        cac=0
        tier1_CAC.append(cac)
    
for i in tier_key:
    try:
        cac=tier2[i]/list(get_last_nondirect(convert2)).count(i)
        tier2_CAC.append(cac)
    except:
        cac=0
        tier2_CAC.append(cac)    
        
for i in tier_key:
    try:
        cac=tier3[i]/list(get_last_nondirect(convert3)).count(i)
        tier3_CAC.append(cac)
    except:
        cac=0
        tier3_CAC.append(cac)    

for i in range(len(tier_key)):
    ave=(tier1_CAC[i]+tier2_CAC[i]+tier3_CAC[i])/3
    average_chanel.append(ave)
    

In [110]:
Lastnondirect_CAC=pd.DataFrame([tier1_CAC,tier2_CAC,tier3_CAC,average_chanel],
                               index=['tier1','tier2','tier3','average'],
                               columns=tier_key)
Lastnondirect_CAC.to_csv('Lastnondirect_CAC.csv')
print(Lastnondirect_CAC)

            email    social   display  paid_search  referral  organic_search  \
tier1    3.344482  1.607717  1.883239     5.555556  0.841043             0.0   
tier2    3.759398  1.870907  2.427184     5.882353  0.941620             0.0   
tier3    4.373178  2.386635  3.128259     6.198347  1.123175             0.0   
average  3.825686  1.955086  2.479561     5.878752  0.968612             0.0   

         direct  
tier1       0.0  
tier2       0.0  
tier3       0.0  
average     0.0  
